In [1]:
import glob
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.features.feature_metadata import FeatureMetadata

In [2]:
model_name = "play_model_1-best-3"
training_files = 3
data_files = glob.glob('play_data/play_model_1-20240906*.tsv')
dfs = [pd.read_csv(file, sep='\t') for file in data_files[:training_files]]

train_data = TabularDataset(pd.concat(dfs, ignore_index=True))
train_data['play_pos'] = train_data['play_pos'].astype('category')

In [3]:
train_data.head()

,run_id,key,play_pos,pos,partner_winning,lead_trumped,tricks_won,trumps_seen,turn_seen,next_seen,...,throw_off_void,throw_off_long,throw_off_low,throw_off_next,tricks_min,tricks_max,tricks_avg,points_min,points_max,points_avg
0,77127b,9♦ J♦ 10♣ Q♣ Q♠,23,0,0,0,2,7,7,4,...,0,0,0,0,2,2,2.0,-1,-1,-1.0
1,77127b,9♦ J♦ 10♣ Q♠ Q♣,23,0,0,0,2,7,7,4,...,0,0,0,0,2,2,2.0,-1,-1,-1.0
2,77127b,10♣ 9♦ J♦ Q♠ Q♣,23,0,0,0,2,7,7,4,...,0,0,0,0,2,2,2.0,-1,-1,-1.0
3,77127b,10♣ 9♦ J♦ Q♣ Q♠,20,0,0,0,3,7,7,3,...,0,0,0,0,3,3,3.0,2,2,2.0
4,77127b,9♦ J♦ Q♠ 10♣ Q♣,23,0,0,0,2,7,7,4,...,0,0,0,0,2,2,2.0,-1,-1,-1.0


In [4]:
label = 'points_avg'
problem_type = 'regression'
presets = 'best_quality'
train_data[label].describe()

count    575696.000000
mean         -0.440327
std           1.925021
min          -4.000000
25%          -2.000000
50%          -1.000000
75%           1.000000
max           4.000000
Name: points_avg, dtype: float64

In [5]:
exclude = ['run_id', 'key']
outputs = ['tricks_min', 'tricks_max', 'tricks_avg', 'points_min', 'points_max', 'points_avg']
assert label in outputs
outputs.remove(label)

predictor = TabularPredictor(problem_type=problem_type, label=label, path=f"models/{model_name}")
predictor.fit(train_data.drop(columns=(exclude + outputs)), presets=presets)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.9
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #202405300957~1722440358~24.04~aa0a43f~dev-Ubuntu SMP PREEMPT_DY
CPU Count:          8
Memory Avail:       10.74 GB / 15.34 GB (70.1%)
Disk Space Avail:   116.59 GB / 452.95 GB (25.7%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to

(_ray_fit pid=18487) [1000]	valid_set's rmse: 0.865577


(_dystack pid=17484) 	-0.8043	 = Validation score   (-root_mean_squared_error)
(_dystack pid=17484) 	173.75s	 = Training   runtime
(_dystack pid=17484) 	647.19s	 = Validation runtime
(_dystack pid=17484) Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.0s of the 252.78s of remaining time.
(_dystack pid=17484) 	Ensemble Weights: {'KNeighborsUnif_BAG_L1': 0.667, 'LightGBMXT_BAG_L1': 0.333}
(_dystack pid=17484) 	-0.6549	 = Validation score   (-root_mean_squared_error)
(_dystack pid=17484) 	0.41s	 = Training   runtime
(_dystack pid=17484) 	0.0s	 = Validation runtime
(_dystack pid=17484) Fitting 106 L2 models ...
(_dystack pid=17484) Fitting model: LightGBMXT_BAG_L2 ... Training model for up to 252.35s of the 252.31s of remaining time.
(_dystack pid=17484) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=6.98%)


(_ray_fit pid=19088) [1000]	valid_set's rmse: 0.56966 [repeated 8x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=19090) [2000]	valid_set's rmse: 0.560787 [repeated 8x across cluster]


(_dystack pid=17484) 	-0.5572	 = Validation score   (-root_mean_squared_error)
(_dystack pid=17484) 	204.08s	 = Training   runtime
(_dystack pid=17484) 	669.03s	 = Validation runtime
(_dystack pid=17484) Fitting model: WeightedEnsemble_L3 ... Training model for up to 360.0s of the -40.4s of remaining time.
(_dystack pid=17484) 	Ensemble Weights: {'LightGBMXT_BAG_L2': 1.0}
(_dystack pid=17484) 	-0.5572	 = Validation score   (-root_mean_squared_error)
(_dystack pid=17484) 	0.34s	 = Training   runtime
(_dystack pid=17484) 	0.0s	 = Validation runtime
(_dystack pid=17484) AutoGluon training complete, total runtime = 936.99s ... Best model: WeightedEnsemble_L3 | Estimated inference throughput: 46.9 rows/s (63967 batch size)
(_dystack pid=17484) TabularPredictor saved. To load, use: predictor = TabularPredictor.load("models/play_model_1-best-3/ds_sub_fit/sub_fit_ho")
(_dystack pid=17484) Deleting DyStack predictor artifacts (clean_up_fits=True) ...
Leaderboard on holdout data (DyStack):
     

In [6]:
test_data = TabularDataset(data_files[5])[:100000]
y_pred = predictor.predict(test_data[:1000])
y_pred.head(10)

Loaded data from: play_data/play_model_1-20240906_173932.tsv | Columns = 55 / 55 | Rows = 194348 -> 194348


0   -1.414442
1   -1.317378
2   -1.310388
3   -1.414442
4   -1.310388
5   -1.317378
6   -1.414442
7   -1.317378
8   -1.414442
9   -1.310388
Name: points_avg, dtype: float32

In [7]:
predictor.evaluate(test_data, silent=True)

{'root_mean_squared_error': -1.199468738124719,
 'mean_squared_error': -1.438725253738506,
 'mean_absolute_error': -0.7792926110285621,
 'r2': 0.5979880433696261,
 'pearsonr': 0.7785859796534218,
 'median_absolute_error': -0.4315398931503296}

In [8]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L1,-1.001750,-0.703915,root_mean_squared_error,175.608007,2210.794329,452.927264,175.608007,2210.794329,452.927264,1,True,3
1,LightGBMXT_BAG_L2,-1.199469,-0.453420,root_mean_squared_error,583.389504,6127.209040,1000.495010,206.231662,2775.053428,546.852738,2,True,5
2,WeightedEnsemble_L3,-1.199469,-0.453420,root_mean_squared_error,583.393654,6127.214326,1000.920489,0.004150,0.005286,0.425479,3,True,6
3,WeightedEnsemble_L2,-1.305392,-0.583871,root_mean_squared_error,276.780269,2780.292279,453.806600,0.002437,0.005419,0.519767,2,True,4
4,KNeighborsUnif_BAG_L1,-1.689535,-0.703504,root_mean_squared_error,100.380010,571.868752,0.355439,100.380010,571.868752,0.355439,1,True,1
5,KNeighborsDist_BAG_L1,-1.693906,-0.638449,root_mean_squared_error,101.169825,569.492531,0.359569,101.169825,569.492531,0.359569,1,True,2
